In [ ]:
import pandas as pd
from geopy.distance import geodesic
import numpy as np
from tqdm import tqdm

In [ ]:
# --- 1. Load Initial Data ---
print("Loading data...")
train_df = pd.read_csv('train_cleaned_c-10-21.csv')
details_df = pd.read_csv('auxiliary-data/sg-hdb-block-details.csv')

original_count = len(train_df)
print(f"Original 'train.csv' data has {original_count} rows.")
print("-" * 30)

# --- 2. Clean and Prepare Data for Merging ---
# For safety, we operate on copies
train_clean_df = train_df.copy()
details_clean_df = details_df.copy()

# Convert key fields ('town'/'TOWN', 'block'/'BLOCK') in both DataFrames
# to lowercase strings and strip leading/trailing whitespace to create join keys
train_clean_df['join_key_town'] = train_clean_df['TOWN'].str.lower().str.strip()
train_clean_df['join_key_block'] = train_clean_df['BLOCK'].astype(str).str.lower().str.strip()

details_clean_df['join_key_town'] = details_clean_df['TOWN'].str.lower().str.strip()
details_clean_df['join_key_block'] = details_clean_df['BLOCK'].astype(str).str.lower().str.strip()
print("Data cleaning complete.")
print("-" * 30)


In [ ]:
# --- 3. Merge Training Data with Block Details (to get coordinates) ---
print("Starting data merge...")
# Perform a left merge, using train_clean_df as the left table
# how='left' means every row from train_clean_df will be kept
merged_df = pd.merge(
    train_clean_df,
    details_clean_df,
    on=['join_key_town', 'join_key_block'],  # Match using the standardized keys we created
    how='left'
)
merged_df.to_csv('merged_latitude.csv', index=False)
print(f"Total rows after merge: {len(merged_df)}")
print("-" * 30)

# --- 4. Load Amenity Data ---
print("Loading amenity datasets...")
merged_df = pd.read_csv('merged_latitude.csv')
subfolder = 'auxiliary-data'
hawkers_df = pd.read_csv(subfolder + '/' + 'sg-gov-hawkers.csv')
mrt_df = pd.read_csv(subfolder + '/' + 'sg-mrt-stations.csv')
primary_schools_df = pd.read_csv(subfolder + '/' + 'sg-primary-schools.csv')
print("Amenity data loaded.")

In [ ]:
# --- 5. Define Distance Calculation Function ---
def calculate_distance_features(main_df, amenities_df, radius_km=1.0, description="Processing"):
    """
    For each row in main_df, calculate distances to all points in amenities_df and extract features.
    (Includes a tqdm progress bar)

    Args:
    - main_df: DataFrame containing properties and their coordinates ('LATITUDE', 'LONGITUDE').
    - amenities_df: DataFrame containing amenities and their coordinates ('LATITUDE', 'LONGITUDE').
    - radius_km: The radius (in km) used to count amenities within range.
    - description: Description text for the progress bar.

    Returns:
    - A tuple of two Series: (min_distance, count_within_radius).
    """
    
    min_distances = []
    count_within_radius = []

    # Wrap the iterator with tqdm in the loop
    # It's good practice to provide the 'total' parameter when using .iterrows()
    iterator = tqdm(main_df.iterrows(), total=main_df.shape[0], desc=description)

    for house_index, house in iterator:
        # Note: The coordinate columns might have suffixes like _x or _y after merging
        house_coords = (house['LATITUDE'], house['LONGITUDE']) 
        
        if pd.isna(house_coords[0]) or pd.isna(house_coords[1]):
            min_distances.append(np.nan)
            count_within_radius.append(np.nan)
            continue

        distances = [
            geodesic(house_coords, (amenity['LATITUDE'], amenity['LONGITUDE'])).kilometers
            for amenity_index, amenity in amenities_df.iterrows()
        ]
        
        if not distances:
            min_distances.append(np.nan)
            count_within_radius.append(0)
        else:
            min_distances.append(min(distances))
            count_within_radius.append(sum(1 for d in distances if d <= radius_km))
            
    return pd.Series(min_distances, index=main_df.index), pd.Series(count_within_radius, index=main_df.index)


In [ ]:
# --- 6. Calculate and Add Amenity Features ---
print("Calculating distance features for MRT stations...")
# Calculate nearest distance to MRT and count of MRT stations within 1km
dist_mrt, num_mrt = calculate_distance_features(merged_df, mrt_df, radius_km=1.0, description="Calculating MRT")
merged_df['dist_to_nearest_mrt'] = dist_mrt
merged_df['num_mrt_within_1km'] = num_mrt
print("MRT station features calculation complete.")

print("Calculating distance features for Hawker Centres...")
# Calculate nearest distance to Hawker Centre and count within 1km
dist_hawker, num_hawker = calculate_distance_features(merged_df, hawkers_df, radius_km=1.0, description="Calculating Hawkers")
merged_df['dist_to_nearest_hawker'] = dist_hawker
merged_df['num_hawkers_within_1km'] = num_hawker
print("Hawker Centre features calculation complete.")

print("Calculating distance features for Primary Schools...")
# Calculate nearest distance to Primary School and count within 2km (School districts can be larger)
dist_primary, num_primary = calculate_distance_features(merged_df, primary_schools_df, radius_km=2.0, description="Calculating Schools")
merged_df['dist_to_nearest_primary_school'] = dist_primary
merged_df['num_primary_schools_within_2km'] = num_primary
print("Primary School features calculation complete.")
print("-" * 30)

merged_df.to_csv('merged_with_amenities.csv', index=False)


In [ ]:
# --- 7. Final Cleanup and Saving ---
print("All features added. Cleaning up final DataFrame...")
df = pd.read_csv("merged_with_amenities.csv")

# Drop intermediate, redundant, or sensitive columns
df = df.drop(columns=[
    'month', 'TOWN_x', 'BLOCK_x', 'join_key_town', 'join_key_block',
    'TOWN_y', 'BLOCK_y', 'ADDRESS', 'POSTAL_CODE', 'LATITUDE', 'LONGITUDE'
])
print("Final columns:", df.columns)

# Drop duplicates based on ID, keeping the first entry
df = df.drop_duplicates(subset=['ID'], keep='first')
print(f"Total rows after dropping duplicates: {len(df)}")

# Save the final feature-engineered dataset
df.to_csv("train_cleaned_fe_geo_final.csv", index=False)
print("Processing complete. Final file saved as 'train_cleaned_fe_geo_final.csv'.")